In [80]:
import pymongo
from pymongo import MongoClient
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout, TimeDistributed, Conv2D, MaxPooling2D, Flatten
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.layers import Bidirectional
import tensorflowjs as tfjs

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]

In [3]:
client = MongoClient('localhost', 27017)
db = client.Dataset
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'Dataset')

In [4]:
poses = db.PoseValues

In [5]:
df = pd.DataFrame(list(poses.find()))

In [6]:
df.head()

,_id,word,data
0,620bc2440d16b90ca807d48b,hi,"[{'faceLandmarks': [{'x': 0.5134519338607788, ..."
1,620bc2440d16b90ca807d48c,hi,"[{'faceLandmarks': [{'x': 0.5285636186599731, ..."
2,620bc2440d16b90ca807d48d,hi,"[{'faceLandmarks': [{'x': 0.5248129963874817, ..."
3,620bc28a0d16b90ca807d48f,hi,"[{'faceLandmarks': [{'x': 0.5378047823905945, ..."
4,620bc28a0d16b90ca807d490,hi,"[{'faceLandmarks': [{'x': 0.5376523733139038, ..."


In [7]:
df.shape

(120, 3)

In [8]:
type(df['data'][0][0])

dict

In [9]:
def extract_keypoints(result):
    pose = np.array([[res['x'], res['y'], res['z'], res['visibility']] for res in result["poseLandmarks"]]).flatten() if "poseLandmarks" in result else np.zeros(33*4)
    lh = np.array([[res['x'], res['y'], res['z']] for res in result["leftHandLandmarks"]]).flatten() if "leftHandLandmarks" in result else np.zeros(21*3)
    rh = np.array([[res['x'], res['y'], res['z']] for res in result["rightHandLandmarks"]]).flatten() if "rightHandLandmarks" in result else np.zeros(21*3)
    face = np.array([[res['x'], res['y'], res['z']] for res in result["faceLandmarks"]]).flatten() if "faceLandmarks" in result else np.zeros(468*3)
    # ea = np.array([[res['x'], res['y'], res['z']] for res in result["ea"]]).flatten() if "ea" in result else np.zeros(33*3)
    return np.concatenate([pose, lh, rh, face])

In [10]:
extract_keypoints(df['data'][0][20]).shape

(1662,)

In [11]:
data = []
for videos in df['data']:
    frame_data = []
    for frames in videos:
        frame_data.append(extract_keypoints(frames))
    data.append(frame_data)

In [12]:
dataset = np.array(data)

In [13]:
dataset.shape

(120, 40, 1662)

In [14]:
words = np.array(df['word']).flatten()
words.shape

(120,)

In [15]:
label_map = {label:num for num, label in enumerate(np.unique(words))}
label_map

{'hi': 0, 'i': 1, 'name': 2}

In [16]:
labels = [label_map[word] for word in df['word']]
df['word'].shape[0]

120

In [17]:
# with tf.device('/GPU'):
#     a = tf.random.normal(shape=(2,), dtype=tf.float32)
#     b = tf.nn.relu(a)

In [18]:
X = dataset
y = to_categorical(labels).astype(int)

In [19]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [20]:
# with tf.device('/CPU'):
# model = Sequential()
# model.add(Bidirectional(LSTM(64,dropout=0.2, recurrent_dropout=0.2, return_sequences=True, activation='relu', input_shape=(40, 1761))))
# model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu')))
# model.add(Bidirectional(LSTM(64, return_sequences=True, activation='relu')))
# model.add(Bidirectional(LSTM(128,dropout=0.2, recurrent_dropout=0.2, return_sequences=True, activation='relu')))
# # model.add(LSTM(64, return_sequences=True, activation='relu'))
# # model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(Bidirectional(LSTM(64, return_sequences=False, activation='relu')))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
# model.add(Dense(2, activation='softmax'))
# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [54]:
model = Sequential()
# model.add(TimeDistributed(Conv2D(1, (2,2), activation='relu', padding='same', input_shape=(40,1662,1))))
# model.add(TimeDistributed(MaxPooling2D(pool_size=(2, 2))))
# model.add(TimeDistributed(Flatten()))
model.add(LSTM(100, input_shape=(40, 1662)))
model.add(Dropout(0.5))
model.add(Dense(100, activation='relu'))
# model.add(LSTM(64,return_sequences=True, activation='relu', input_shape=(40, 1662)))
# model.add(LSTM(128, return_sequences=True, activation='relu'))
# model.add(LSTM(64, return_sequences=False, activation='relu'))
# model.add(Dense(64, activation='relu'))
# model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [55]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [56]:
X_test.shape


(6, 40, 1662)

In [57]:
y_test.shape

(6, 3)

In [58]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
4/4 [==============================] - 2s 61ms/step - loss: 1.1491 - categorical_accuracy: 0.3509
Epoch 2/2000
4/4 [==============================] - 0s 55ms/step - loss: 1.0744 - categorical_accuracy: 0.3947
Epoch 3/2000
4/4 [==============================] - 0s 71ms/step - loss: 1.0764 - categorical_accuracy: 0.4035
Epoch 4/2000
4/4 [==============================] - 0s 69ms/step - loss: 1.0298 - categorical_accuracy: 0.4211
Epoch 5/2000
4/4 [==============================] - 0s 64ms/step - loss: 1.0397 - categorical_accuracy: 0.4561
Epoch 6/2000
4/4 [==============================] - 0s 62ms/step - loss: 0.9932 - categorical_accuracy: 0.5088
Epoch 7/2000
4/4 [==============================] - 0s 66ms/step - loss: 0.9221 - categorical_accuracy: 0.5263
Epoch 8/2000
4/4 [==============================] - 0s 72ms/step - loss: 0.8853 - categorical_accuracy: 0.5702
Epoch 9/2000
4/4 [==============================] - 0s 70ms/step - loss: 0.8586 - categorical_accuracy: 0.6228
E

KeyboardInterrupt: 

In [59]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 100)               705200    
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 100)               10100     
                                                                 
 dense_12 (Dense)            (None, 3)                 303       
                                                                 
Total params: 715,603
Trainable params: 715,603
Non-trainable params: 0
_________________________________________________________________


In [60]:
res = model.predict(X_test)
np.argmax(res[5])

2

In [61]:
y_test

array([[0, 0, 1],
       [0, 0, 1],
       [0, 0, 1],
       [1, 0, 0],
       [1, 0, 0],
       [0, 0, 1]])

In [62]:
accr = model.evaluate(X_test,y_test)
print('Test set\n  Loss: {:0.3f}\n  Accuracy: {:0.3f}'.format(accr[0],accr[1]))

1/1 [==============================] - 0s 363ms/step - loss: 3.7332e-04 - categorical_accuracy: 1.0000
Test set
  Loss: 0.000
  Accuracy: 1.000


In [63]:
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score
yhat = model.predict(X_test)
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue, yhat)

array([[[4, 0],
        [0, 2]],

       [[2, 0],
        [0, 4]]])

In [44]:
accuracy_score(ytrue, yhat)

1.0

In [68]:
model.save('SLD.h5')

In [46]:
del model

In [69]:
loaded_model = load_model('SLD.h5')

In [70]:
loaded_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_10 (LSTM)              (None, 100)               705200    
                                                                 
 dropout_1 (Dropout)         (None, 100)               0         
                                                                 
 dense_11 (Dense)            (None, 100)               10100     
                                                                 
 dense_12 (Dense)            (None, 3)                 303       
                                                                 
Total params: 715,603
Trainable params: 715,603
Non-trainable params: 0
_________________________________________________________________


In [79]:
converter = tf.lite.TFLiteConverter.from_keras_model(loaded_model)
# tflite_model = converter.convert()
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.experimental_new_converter=True
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS,
tf.lite.OpsSet.SELECT_TF_OPS]

tfmodel = converter.convert()
open('image_model.tflite', 'wb').write(tfmodel)

INFO:tensorflow:Assets written to: /var/folders/j5/v6fyg4pd095c4c97hw6gzbdr0000gn/T/tmpxn9gkmrh/assets


INFO:tensorflow:Assets written to: /var/folders/j5/v6fyg4pd095c4c97hw6gzbdr0000gn/T/tmpxn9gkmrh/assets
2022-02-15 15:14:41.845365: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:357] Ignored output_format.
2022-02-15 15:14:41.845383: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:360] Ignored drop_control_dependency.
2022-02-15 15:14:41.845490: I tensorflow/cc/saved_model/reader.cc:43] Reading SavedModel from: /var/folders/j5/v6fyg4pd095c4c97hw6gzbdr0000gn/T/tmpxn9gkmrh
2022-02-15 15:14:41.854530: I tensorflow/cc/saved_model/reader.cc:78] Reading meta graph with tags { serve }
2022-02-15 15:14:41.854549: I tensorflow/cc/saved_model/reader.cc:119] Reading SavedModel debug info (if present) from: /var/folders/j5/v6fyg4pd095c4c97hw6gzbdr0000gn/T/tmpxn9gkmrh
2022-02-15 15:14:41.899139: I tensorflow/cc/saved_model/loader.cc:228] Restoring SavedModel bundle.
2022-02-15 15:14:41.999773: I tensorflow/cc/saved_model/loader.cc:212] Running initialization

729968

In [81]:
tfjs.converters.save_keras_model(loaded_model, "./jsModel")